# 0. 패키지 불러오기

In [1]:
!pip install catboost
!pip install category_encoders
!pip install optuna
!pip install kmodes
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 1.7 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.4
    Uninstalling scipy-1.5.4:
      Successfully uninstalled scipy-1.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
pycaret 2.3.10 requires scipy<=1.5.4, but you have scipy 1.7.3 which is incompatible.
jaxlib 0.3.22+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jax 0.3.23 requires numpy>=1.20, but you h

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import math

from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import * # KFold, cross_val_score, train_test_split
from sklearn.metrics import *  # make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import * # LabelEncoder, StandardScaler
from sklearn.svm import * # SVC, SVR
from sklearn.inspection import *
from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import * # PCA

from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from pycaret.classification import *
import random

from lightgbm import *
import lightgbm as lgb
import xgboost as xgb
from datetime import datetime
import calendar
from collections import *

import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

# 1. 데이터 확인

In [3]:
# 구글 드라이브 입력
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls '/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/'

data_info.xlsx	sample_submission.csv  test.csv  train.csv


In [5]:
path = '/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/'

Rdata_train = pd.read_csv(path + 'train.csv')
Rdata_test = pd.read_csv(path + 'test.csv')

print(Rdata_train.shape) # (14095, 54) -> 생각보다 데이터 수가 적다.
print(Rdata_train.size)
print(Rdata_test.shape) # (6041, 19)
print(Rdata_test.size)
Rdata_train.head()

(14095, 54)
761130
(6041, 19)
114779


,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0,1,0,0,1,0,572,2,19,1117,0,0,0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0,4,1,0,0,0,906,1,44,21774,2,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


1. COMPONENT_ARBITRARY : 샘플 오일 관련 부품 (Component 4종, 비식별화) : 명목형
2. ANONYMOUS_1 : 무명 Feautre 1. 수치형 데이터
3. YEAR : 오일 샘플 및 진단 해 (Year)
4. SAMPLE_TRANSFER_DAY : 오일 샘플링 후 진단 기관으로 이동한 기간 (Days)
5. ANONYMOUS_2 : 무명 Feautre 2. 수치형 데이터
6. AG : 원소기호 AG - Silver 함유량
7. AL : 원소기호 AL - Aluminium 함유량
8. B : 원소기호 B - Boron 함유량
9. BA : 원소기호 BA -  Barium 함유량
10. BE : 원소기호 BE - Beryllium 함유량
11. CA : 원소기호 CA - Calcium 함유량
12. CD : 원소기호 CD - Cadmium 함유량
13. CO : 원소기호 CO - Cobolt 함유량
14. CR : 원소기호 CR - Chromium 함유량
15. CU : 원소기호 CU - Copper 함유량
16. FH2O : Water 수치(By FT-IR)
17. FNOX : NOx 수치(By FT-IR)
18. FOPTIMETHGLY : 비식별화
19. FOXID : Oxidation 수치(By FT-IR)
20. FSO4 : SO4 수치(By FT-IR)
21. FTBN : 염기성 첨가제물질 수치(By FT-IR)
22. FE : 원소기호 FE - Iron 함유량
23. FUEL : 연료 함유량
24. H2O : Water 함유량
25. K : 원소기호 K - Potassium 함유량
26. LI : 원소기호 LI - Lithium 함유량
27. MG : 원소기호 MG - Magnesium 함유량
28. MN : 원소기호 MN - Manganese 함유량
29. MO : 원소기호 MO - Molybdenum 함유량
30. NA : 원소기호 NA - Sodium 함유량
31. NI : 원소기호 NI - Nickel 함유량
32. P : 원소기호 P - Phosphorus 함유량
33. PB : 원소기호 PB - Lead 함유량
34. PQINDEX : Particle Quantifier Index
35. S : 원소기호 S - Sulphur 함유량
36. SB : 원소기호 SB - Antimony 함유량
37. SI : 원소기호 SI - Silicone 함유량
38. SN : 원소기호 SN - Tin 함유량
39. SOOTPERCENTAGE : Soot 함유량(%)
40. TI : 원소기호 TI - Titanium 함유량
41. U100 : Particle Count (100㎛ 이상 크기 입자)
42. U75 : Particle Count (75㎛ 이상 크기 입자)
43. U50 : Particle Count (50㎛ 이상 크기 입자)
44. U25 : Particle Count (25㎛ 이상 크기 입자)
45. U20 : Particle Count (20㎛ 이상 크기 입자)
46. U14 : Particle Count (14㎛ 이상 크기 입자)
47. U6 : Particle Count (6㎛ 이상 크기 입자)
48. U4 : Particle Count (4㎛ 이상 크기 입자)
49. V : 원소기호 V - Vanadium 함유량
50. V100 : Viscosity @ 100 degrees
51. V40 : Viscosity @ 40 degrees
52. ZN : 원소기호 ZN - Zinc 함유량
53. Y_LABEL : 오일 정상 여부 (0 : 정상, 1 : 이상)

### 준비

In [6]:
Rdata_train.dtypes.value_counts()

# float로 되어있지만 사실 정수형인 변수들이 많다.
# 결측치가 있으면 정수형이 아닌 float로 변환되는 것 같다.

int64      31
float64    21
object      2
dtype: int64

In [7]:
Rdata_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)
# nunique : 고윳값의 개수를 알려준다.
# select_dtypes('int64') : int64인 변수들만 데이터로 불러온다.

ANONYMOUS_1            2951
YEAR                     16
SAMPLE_TRANSFER_DAY     107
ANONYMOUS_2             842
AG                        4
AL                      328
B                       504
BA                       38
BE                        9
CA                     2581
CO                       10
CR                      102
CU                      496
FE                     1281
LI                       36
MG                      554
MN                      100
MO                      298
NA                       83
NI                       57
P                      2191
PB                       73
PQINDEX                1910
S                      8076
SB                       38
SI                      514
SN                       37
TI                       71
V                        15
ZN                     1552
Y_LABEL                   2
dtype: int64

In [8]:
Rdata_train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)
# 결측값은 제거한 채로 고윳값의 개수를 알려준다.

CD                   6
FH2O                72
FNOX                21
FOPTIMETHGLY         8
FOXID               34
FSO4                35
FTBN                99
FUEL                21
H2O                 47
K                  102
SOOTPERCENTAGE      55
U100                14
U75                 20
U50                 90
U25                195
U20                275
U14                549
U6                1766
U4                2053
V100               106
V40               1769
dtype: int64

In [9]:
Rdata_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

ID                     14095
COMPONENT_ARBITRARY        4
dtype: int64

In [10]:
Rdata_train.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR',
       'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA',
       'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI',
       'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100',
       'V40', 'ZN', 'Y_LABEL'],
      dtype='object')

# 2. 18개의 변수만 이용 AutoML

In [ ]:
print('Rdata_train')
print(Rdata_train.dtypes.value_counts())
print('------------------------')
print('Rdata_test')
print(Rdata_test.dtypes.value_counts())

Rdata_train
int64      31
float64    21
object      2
dtype: int64
------------------------
Rdata_test
int64      15
object      2
float64     2
dtype: int64


In [ ]:
print('Rdata_train')
print(Rdata_train.columns)
print('------------------------')
print('Rdata_test')
print(Rdata_test.columns)

Rdata_train
Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR',
       'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA',
       'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI',
       'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100',
       'V40', 'ZN', 'Y_LABEL'],
      dtype='object')
------------------------
Rdata_test
Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')


In [ ]:
AUTO_ML_train1 = Rdata_train.copy()
AUTO_ML_test1 = Rdata_test.copy()

In [ ]:
AUTO_ML_train2 = AUTO_ML_train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL']]
AUTO_ML_test2 = AUTO_ML_test1.drop(['ID'], axis = 1)

print(AUTO_ML_train2.shape)
print(AUTO_ML_test2.shape)

(14095, 19)
(6041, 18)


In [ ]:
print(AUTO_ML_train2.select_dtypes(include = 'object').columns)
print(AUTO_ML_test2.select_dtypes(include = 'object').columns)

Index(['COMPONENT_ARBITRARY'], dtype='object')
Index(['COMPONENT_ARBITRARY'], dtype='object')


In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()

AUTO_ML_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(AUTO_ML_train2['COMPONENT_ARBITRARY'])
AUTO_ML_train2['YEAR_category'] = le2.fit_transform(AUTO_ML_train2['YEAR'])

AUTO_ML_test2['COMPONENT_ARBITRARY_category'] = le1.transform(AUTO_ML_test2['COMPONENT_ARBITRARY'])
AUTO_ML_test2['YEAR_category'] = le2.transform(AUTO_ML_test2['YEAR'])

AUTO_ML_train3 = AUTO_ML_train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
AUTO_ML_test3 = AUTO_ML_test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

print(AUTO_ML_train3.shape)
print(AUTO_ML_test3.shape)

(14095, 19)
(6041, 18)


In [ ]:
from pycaret.classification import *

clf = setup(AUTO_ML_train3, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,2352
1,Target,Y_LABEL
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14095, 19)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
        

In [ ]:
best5 = compare_models(fold = 5, sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9152,0.6637,0.0168,0.4722,0.0320,0.0264,0.0724,1.580
lightgbm,Light Gradient Boosting Machine,0.9148,0.6604,0.0263,0.4524,0.0497,0.0406,0.0924,0.380
ada,Ada Boost Classifier,0.9141,0.6587,0.0048,0.1867,0.0093,0.0054,0.0179,0.494
catboost,CatBoost Classifier,0.9154,0.6564,0.0204,0.5393,0.0391,0.0327,0.0909,6.472
rf,Random Forest Classifier,0.9164,0.6355,0.0192,0.7643,0.0373,0.0331,0.1112,1.554
lda,Linear Discriminant Analysis,0.9140,0.6304,0.0048,0.1586,0.0093,0.0052,0.0162,0.088
et,Extra Trees Classifier,0.9123,0.6269,0.0287,0.3095,0.0524,0.0386,0.0715,1.184
lr,Logistic Regression,0.9149,0.6258,0.0024,0.2000,0.0047,0.0029,0.0147,0.688
nb,Naive Bayes,0.6723,0.5915,0.4395,0.1177,0.1831,0.0592,0.0806,0.050
knn,K Neighbors Classifier,0.9094,0.5796,0.0228,0.2053,0.0408,0.0237,0.0412,1.348


INFO:logs:create_model_container: 45
INFO:logs:master_model_container: 45
INFO:logs:display_container: 4
INFO:logs:[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2352, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'F1')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9164,0.6795,0.0240,0.6667,0.0462,0.0406,0.1155
1,0.9164,0.6671,0.0180,0.7500,0.0351,0.0313,0.1077
2,0.9154,0.6626,0.0060,0.5000,0.0118,0.0099,0.0475
3,0.9164,0.6747,0.0180,0.7500,0.0351,0.0313,0.1077
4,0.9154,0.6673,0.0180,0.5000,0.0347,0.0290,0.0824
Mean,0.9160,0.6702,0.0168,0.6333,0.0326,0.0284,0.0922
Std,0.0005,0.0061,0.0059,0.1130,0.0113,0.0101,0.0250


INFO:logs:create_model_container: 46
INFO:logs:master_model_container: 46
INFO:logs:display_container: 5
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
        

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9144,0.695,0.019,0.875,0.0372,0.0337,0.1217


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])
prections = prep_pipe.predict_proba(AUTO_ML_test3)
prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V

array([[0.86737381, 0.13262619],
       [0.84388428, 0.15611572],
       [0.81298649, 0.18701351],
       ...,
       [0.78522382, 0.21477618],
       [0.85743584, 0.14256416],
       [0.83655732, 0.16344268]])

In [ ]:
scores = []
for threshold in range(1, 20) :
    threshold = threshold / 20
    pred = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

print(scores)
print(np.mean(scores))
# 상당히 낮은 threshold -> 0.2796


0.95
[0.15727546084455485, 0.1575948123403419, 0.34643628509719226, 0.7573182247403211, 0.949003984063745, 0.8889895786135024, 0.6823400765445599, 0.4877435575109994, 0.32567849686847594, 0.2084884586746091, 0.12616822429906543, 0.08286852589641433, 0.0549717057396928, 0.04390243902439024, 0.02786885245901639, 0.014851485148514849, 0.0, 0.0, 0.0]
0.27955264041396816


In [ ]:
answer = np.zeros(prections.shape[0])

for i in range(prections.shape[0]) :
  if prections[i][1] >= 0.27955264041396816 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5851, 1.0: 190})

In [ ]:
# 제출

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_pycaret1.csv', index = False)

# 3. train 데이터에서 AL 사용하여 AutoML(변수 19개) 그리고
#    knowledge distillation(변수 18개)

## (1)  model1

In [ ]:
new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']
# 주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개
# 부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개
# 부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개

In [ ]:
AUTO_ML_train1 = Rdata_train.copy()
AUTO_ML_test1 = Rdata_test.copy()

In [ ]:
AUTO_ML_train2 = AUTO_ML_train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL']]
AUTO_ML_test2 = AUTO_ML_test1.drop(['ID'], axis = 1)

print(AUTO_ML_train2.shape) # 변수 19개
print(AUTO_ML_test2.shape) # 변수 18개

(14095, 20)
(6041, 18)


In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()

AUTO_ML_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(AUTO_ML_train2['COMPONENT_ARBITRARY'])
AUTO_ML_train2['YEAR_category'] = le2.fit_transform(AUTO_ML_train2['YEAR'])

AUTO_ML_test2['COMPONENT_ARBITRARY_category'] = le1.transform(AUTO_ML_test2['COMPONENT_ARBITRARY'])
AUTO_ML_test2['YEAR_category'] = le2.transform(AUTO_ML_test2['YEAR'])

AUTO_ML_train3 = AUTO_ML_train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
AUTO_ML_test3 = AUTO_ML_test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL']

print(AUTO_ML_train3.shape)
print(AUTO_ML_test3.shape)

(14095, 20)
(6041, 18)


In [ ]:
from pycaret.classification import *

clf = setup(AUTO_ML_train3, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,5124
1,Target,Y_LABEL
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14095, 20)"
5,Missing Values,False
6,Numeric Features,17
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
        

In [ ]:
best5 = compare_models(fold = 5, sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9501,0.8627,0.4872,0.9290,0.6388,0.6149,0.6522,7.056
gbc,Gradient Boosting Classifier,0.9494,0.8590,0.4883,0.9135,0.6362,0.6118,0.6467,1.660
lightgbm,Light Gradient Boosting Machine,0.9498,0.8532,0.4883,0.9220,0.6379,0.6137,0.6500,0.412
lr,Logistic Regression,0.9488,0.8451,0.4603,0.9497,0.6194,0.5955,0.6410,1.004
ada,Ada Boost Classifier,0.9492,0.8429,0.4749,0.9315,0.6289,0.6047,0.6446,0.494
rf,Random Forest Classifier,0.9499,0.8390,0.4782,0.9396,0.6336,0.6098,0.6502,2.138
et,Extra Trees Classifier,0.9439,0.8334,0.4000,0.9553,0.5629,0.5385,0.5977,1.184
lda,Linear Discriminant Analysis,0.9237,0.8215,0.1732,0.9249,0.2903,0.2696,0.3800,0.092
nb,Naive Bayes,0.8926,0.8075,0.5765,0.4361,0.4950,0.4366,0.4427,0.078
dt,Decision Tree Classifier,0.9097,0.7416,0.5363,0.5016,0.5183,0.4685,0.4689,0.218


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 2
INFO:logs:[<catboost.core.CatBoostClassifier object at 0x7fba25ea6690>, GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=5124, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learnin

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'AUC')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9544,0.8660,0.5196,0.9588,0.6739,0.6517,0.6872
1,0.9524,0.8526,0.4804,0.9885,0.6466,0.6243,0.6713
2,0.9498,0.8515,0.4693,0.9545,0.6292,0.6056,0.6498
3,0.9442,0.8485,0.4246,0.9157,0.5802,0.5545,0.6019
4,0.9529,0.9006,0.5084,0.9479,0.6618,0.6389,0.6749
Mean,0.9507,0.8638,0.4804,0.9531,0.6383,0.6150,0.6571
Std,0.0036,0.0193,0.0333,0.0233,0.0327,0.0339,0.0301


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7fba25d81ad0>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9631,0.8932,0.5162,0.9578,0.6709,0.6532,0.6884


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7fba25d81ad0>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_spl

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

# prections = prep_pipe.predict_proba(AUTO_ML_test3)
# prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V

In [ ]:
# 이건 그냥 본거

scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157306
5,0.10,0.158258
6,0.12,0.291047
7,0.14,0.476756
8,0.16,0.641982
9,0.18,0.755834


In [ ]:
AUTO_ML_train3['model1_prob'] = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]

## (2)  model2

In [ ]:
AUTO_ML_train4 = AUTO_ML_train3.drop(['Y_LABEL', 'AL'], axis = 1)

print(AUTO_ML_train4.shape)
print(AUTO_ML_test3.shape)

(14095, 19)
(6041, 18)


In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
print(len(categorical_features))
print(len(numeric_features))

2
16


In [ ]:
from pycaret.regression import *

clf = setup(AUTO_ML_train4, 
            target = 'model1_prob', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,411
1,Target,model1_prob
2,Original Data,"(14095, 19)"
3,Missing Values,False
4,Numeric Features,16
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(9866, 36)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                                  

In [ ]:
best5 = compare_models(fold = 5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.0696,0.0241,0.1551,0.0697,0.1062,0.3376,1.506
catboost,CatBoost Regressor,0.0709,0.0243,0.1559,0.0600,0.1075,0.3485,5.444
lightgbm,Light Gradient Boosting Machine,0.0712,0.0246,0.1566,0.0516,0.1080,0.3487,0.380
br,Bayesian Ridge,0.0726,0.0250,0.1580,0.0342,0.1082,0.3588,0.052
lr,Linear Regression,0.0726,0.0250,0.1581,0.0337,0.1083,0.3567,0.610
ridge,Ridge Regression,0.0726,0.0250,0.1581,0.0337,0.1083,0.3567,0.040
lar,Least Angle Regression,0.0728,0.0251,0.1584,0.0301,0.1086,0.3589,0.050
omp,Orthogonal Matching Pursuit,0.0735,0.0254,0.1593,0.0186,0.1092,0.3646,0.040
rf,Random Forest Regressor,0.0753,0.0255,0.1595,0.0167,0.1110,0.3881,7.736
llar,Lasso Least Angle Regression,0.0763,0.0259,0.1609,-0.0006,0.1107,0.3917,0.040


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=411, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), <catboost.core.CatBoostRegressor object at 0x7fba25ef9d90>, LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
         

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0680,0.0220,0.1485,0.0653,0.1018,0.3448
1,0.0697,0.0236,0.1537,0.0640,0.1054,0.3458
2,0.0705,0.0250,0.1582,0.0682,0.1078,0.3336
3,0.0739,0.0268,0.1636,0.0896,0.1113,0.3379
4,0.0679,0.0221,0.1487,0.0950,0.1018,0.3378
Mean,0.0700,0.0239,0.1546,0.0764,0.1056,0.3400
Std,0.0022,0.0018,0.0058,0.0131,0.0036,0.0046


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
     

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0728,0.0255,0.1598,0.0679,0.1092,0.3486


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                             

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

prections = prep_pipe.predict(AUTO_ML_test3)
prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                     

array([0.13792819, 0.17255511, 0.20165124, ..., 0.17664178, 0.15825579,
       0.12406229])

In [ ]:
scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict(AUTO_ML_train4)
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

# 0.22

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157296
5,0.10,0.157523
6,0.12,0.161355
7,0.14,0.194231
8,0.16,0.263536
9,0.18,0.344324


In [ ]:
answer = np.zeros(prections.shape[0])

for i in range(prections.shape[0]) :
  if prections[i] >= 0.22 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5614, 1.0: 427})

In [ ]:
# 제출

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_pycaret2.csv', index = False)

# 4. train 데이터에서 AL 추가하고,  
# 앞에서 유의하다고 구한 변수 12개 AutoML 그리고
#    knowledge distillation(변수 11개)

## (1)  model1

In [ ]:
new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']
# 주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개
# 부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개
# 부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개

In [ ]:
AUTO_ML_train1 = Rdata_train.copy()
AUTO_ML_test1 = Rdata_test.copy()

In [ ]:
main_val_12 = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 
            'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN', 'AL', 'Y_LABEL']
main_val_11 = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 
               'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN']

In [ ]:
AUTO_ML_train2 = AUTO_ML_train1.loc[:, main_val_12]
AUTO_ML_test2 = AUTO_ML_test1.loc[:, main_val_11]

print(AUTO_ML_train2.shape) # 변수 12개
print(AUTO_ML_test2.shape) # 변수 11개

(14095, 13)
(6041, 11)


In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()

AUTO_ML_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(AUTO_ML_train2['COMPONENT_ARBITRARY'])
AUTO_ML_train2['YEAR_category'] = le2.fit_transform(AUTO_ML_train2['YEAR'])

AUTO_ML_test2['COMPONENT_ARBITRARY_category'] = le1.transform(AUTO_ML_test2['COMPONENT_ARBITRARY'])
AUTO_ML_test2['YEAR_category'] = le2.transform(AUTO_ML_test2['YEAR'])

AUTO_ML_train3 = AUTO_ML_train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
AUTO_ML_test3 = AUTO_ML_test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN', 'AL']

print(AUTO_ML_train3.shape)
print(AUTO_ML_test3.shape)

(14095, 13)
(6041, 11)


In [ ]:
from pycaret.classification import *

clf = setup(AUTO_ML_train3, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,6480
1,Target,Y_LABEL
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14095, 13)"
5,Missing Values,False
6,Numeric Features,10
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'CR',
                                                          'CU', 'FE', 'NI',
                                                          'PQINDEX', 'V40',
                                                          'ZN', 'AL'],
                                      target='Y_LABEL', time_features=[])),
                ('i

In [ ]:
best5 = compare_models(fold = 5, sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9549,0.8617,0.5086,0.9052,0.6508,0.6287,0.6589,1.466
catboost,CatBoost Classifier,0.9566,0.8590,0.5123,0.9333,0.6612,0.6401,0.6733,7.388
rf,Random Forest Classifier,0.9565,0.8559,0.4988,0.9531,0.6543,0.6336,0.6719,1.414
ada,Ada Boost Classifier,0.9567,0.8503,0.5037,0.9485,0.6578,0.6371,0.6737,0.468
lightgbm,Light Gradient Boosting Machine,0.9562,0.8483,0.5086,0.9302,0.6574,0.6361,0.6695,0.374
lr,Logistic Regression,0.9553,0.8460,0.4768,0.9657,0.6376,0.6167,0.6610,0.698
et,Extra Trees Classifier,0.9498,0.8387,0.4057,0.9711,0.5710,0.5492,0.6094,1.052
nb,Naive Bayes,0.9247,0.8238,0.5675,0.5464,0.5559,0.5148,0.5154,0.052
lda,Linear Discriminant Analysis,0.9307,0.8156,0.1728,0.9505,0.2902,0.2719,0.3865,0.084
dt,Decision Tree Classifier,0.9187,0.7500,0.5478,0.5090,0.5274,0.4830,0.4836,0.116


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=6480, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), <catboost.core.CatBoostClassifier object at 0x7fba1b6a7450>, RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=N

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'AUC')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9519,0.8660,0.4695,0.9059,0.6185,0.5955,0.6324
1,0.9559,0.8633,0.4785,0.9750,0.6420,0.6214,0.6664
2,0.9564,0.8632,0.4969,0.9529,0.6532,0.6324,0.6708
3,0.9579,0.8746,0.5337,0.9255,0.6770,0.6563,0.6848
4,0.9620,0.8678,0.5521,0.9783,0.7059,0.6872,0.7195
Mean,0.9568,0.8670,0.5062,0.9475,0.6593,0.6386,0.6748
Std,0.0033,0.0042,0.0318,0.0281,0.0300,0.0312,0.0282


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
        

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9482,0.8651,0.4703,0.9286,0.6244,0.5997,0.64


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

# prections = prep_pipe.predict_proba(AUTO_ML_test3)
# prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'CR',
                                                          'CU', 'FE', 'NI',
                                                          'PQINDEX', 'V40',
                                                          'ZN', 'AL'],
                                      target='Y_LABEL', time_features=[])),
      

In [ ]:
# 이건 그냥 본거

scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157306
5,0.10,0.158258
6,0.12,0.291047
7,0.14,0.476756
8,0.16,0.641982
9,0.18,0.755834


In [ ]:
AUTO_ML_train3['model1_prob'] = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]

## (2)  model2

In [ ]:
AUTO_ML_train4 = AUTO_ML_train3.drop(['Y_LABEL', 'AL'], axis = 1)

print(AUTO_ML_train4.shape)
print(AUTO_ML_test3.shape)

(14095, 12)
(6041, 11)


In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN']
print(len(categorical_features))
print(len(numeric_features))

2
9


In [ ]:
from pycaret.regression import *

clf = setup(AUTO_ML_train4, 
            target = 'model1_prob', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,5745
1,Target,model1_prob
2,Original Data,"(14095, 12)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(9866, 29)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'CR',
                                                          'CU', 'FE', 'NI',
                                                          'PQINDEX', 'V40',
                                                          'ZN'],
                                      target='model1_prob', time_features=[])),
                ('imputer',
                 Simp...
          

In [ ]:
best5 = compare_models(fold = 5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.0777,0.0259,0.1605,0.0789,0.1109,0.3857,1.312
lightgbm,Light Gradient Boosting Machine,0.0783,0.0262,0.1617,0.0658,0.1123,0.3899,0.206
catboost,CatBoost Regressor,0.0788,0.0263,0.1618,0.0636,0.1126,0.3952,4.726
rf,Random Forest Regressor,0.0828,0.0271,0.1645,0.0329,0.1153,0.4327,7.456
br,Bayesian Ridge,0.0823,0.0275,0.1655,0.0212,0.1142,0.4212,0.050
lr,Linear Regression,0.0822,0.0275,0.1656,0.0198,0.1143,0.4188,0.050
ridge,Ridge Regression,0.0822,0.0275,0.1656,0.0198,0.1143,0.4188,0.042
ada,AdaBoost Regressor,0.0793,0.0275,0.1656,0.0190,0.1147,0.3993,0.180
omp,Orthogonal Matching Pursuit,0.0835,0.0279,0.1667,0.0064,0.1154,0.4314,0.042
et,Extra Trees Regressor,0.0823,0.0280,0.1671,0.0006,0.1176,0.4322,3.042


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=5745, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_spli

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0749,0.0229,0.1512,0.1074,0.1051,0.3879
1,0.0750,0.0221,0.1485,0.0786,0.1034,0.4062
2,0.0835,0.0302,0.1737,0.0604,0.1189,0.3901
3,0.0800,0.0261,0.1616,0.0964,0.1117,0.3997
4,0.0801,0.0268,0.1637,0.0963,0.1129,0.3921
Mean,0.0787,0.0256,0.1598,0.0878,0.1104,0.3952
Std,0.0033,0.0029,0.0091,0.0165,0.0056,0.0068


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
     

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0813,0.0276,0.1661,0.0785,0.1143,0.3973


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                             

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

prections = prep_pipe.predict(AUTO_ML_test3)
prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'CR',
                                                          'CU', 'FE', 'NI',
                                                          'PQINDEX', 'V40',
                                                          'ZN'],
                                      target='model1_prob', time_features=[])),
                ('imputer',
                 Simp.

array([0.13353462, 0.17699235, 0.19914515, ..., 0.225624  , 0.15542432,
       0.12314356])

In [ ]:
scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict(AUTO_ML_train4)
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

# 0.22

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157275
5,0.10,0.157481
6,0.12,0.169425
7,0.14,0.227527
8,0.16,0.333763
9,0.18,0.469936


In [ ]:
answer = np.zeros(prections.shape[0])

for i in range(prections.shape[0]) :
  if prections[i] >= 0.24 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5640, 1.0: 401})

In [ ]:
# 제출

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_pycaret3.csv', index = False)

# 5. train 데이터에서 AL, BA 사용하여 AutoML(변수 20개) 그리고
#    knowledge distillation(변수 18개)

## (1)  model1

In [ ]:
new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']
# 주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개
# 부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개
# 부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개

In [ ]:
AUTO_ML_train1 = Rdata_train.copy()
AUTO_ML_test1 = Rdata_test.copy()

In [ ]:
AUTO_ML_train2 = AUTO_ML_train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
AUTO_ML_test2 = AUTO_ML_test1.drop(['ID'], axis = 1)

print(AUTO_ML_train2.shape) # 변수 20개
print(AUTO_ML_test2.shape) # 변수 18개

(14095, 21)
(6041, 18)


In [ ]:
le1 = LabelEncoder()
le2 = LabelEncoder()

AUTO_ML_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(AUTO_ML_train2['COMPONENT_ARBITRARY'])
AUTO_ML_train2['YEAR_category'] = le2.fit_transform(AUTO_ML_train2['YEAR'])

AUTO_ML_test2['COMPONENT_ARBITRARY_category'] = le1.transform(AUTO_ML_test2['COMPONENT_ARBITRARY'])
AUTO_ML_test2['YEAR_category'] = le2.transform(AUTO_ML_test2['YEAR'])

AUTO_ML_train3 = AUTO_ML_train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
AUTO_ML_test3 = AUTO_ML_test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL', 'BA']

print(AUTO_ML_train3.shape)
print(AUTO_ML_test3.shape)

(14095, 21)
(6041, 18)


In [ ]:
from pycaret.classification import *

clf = setup(AUTO_ML_train3, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,2874
1,Target,Y_LABEL
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14095, 22)"
5,Missing Values,False
6,Numeric Features,19
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
        

In [ ]:
best5 = compare_models(fold = 5, sort = 'AUC', n_select = 5) # F1으로 정렬

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9520,0.8669,0.4929,0.9057,0.6380,0.6147,0.6476,1.524
catboost,CatBoost Classifier,0.9535,0.8636,0.4929,0.9354,0.6452,0.6228,0.6599,8.836
lightgbm,Light Gradient Boosting Machine,0.9527,0.8577,0.4929,0.9195,0.6415,0.6186,0.6534,0.440
ada,Ada Boost Classifier,0.9513,0.8519,0.4658,0.9360,0.6219,0.5990,0.6410,0.586
lr,Logistic Regression,0.9511,0.8496,0.4552,0.9504,0.6156,0.5929,0.6391,1.406
rf,Random Forest Classifier,0.9529,0.8474,0.4799,0.9448,0.6363,0.6139,0.6545,1.464
et,Extra Trees Classifier,0.9479,0.8457,0.4057,0.9723,0.5722,0.5494,0.6097,1.170
lda,Linear Discriminant Analysis,0.9258,0.8169,0.1462,0.9376,0.2522,0.2347,0.3531,0.076
nb,Naive Bayes,0.8431,0.8168,0.6345,0.3238,0.4176,0.3420,0.3732,0.050
dt,Decision Tree Classifier,0.9138,0.7424,0.5354,0.4985,0.5161,0.4689,0.4693,0.100


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=8671, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), <catboost.core.CatBoostClassifier object at 0x7f85407122d0>, LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learnin

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'AUC')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9504,0.8482,0.4412,0.9615,0.6048,0.5822,0.6329
1,0.9493,0.8739,0.4615,0.8966,0.6094,0.5852,0.6223
2,0.9569,0.8760,0.5030,0.9884,0.6667,0.6462,0.6886
3,0.9549,0.8826,0.4941,0.9655,0.6537,0.6322,0.6731
4,0.9539,0.8618,0.4882,0.9540,0.6459,0.6240,0.6643
Mean,0.9531,0.8685,0.4776,0.9532,0.6361,0.6140,0.6562
Std,0.0028,0.0122,0.0229,0.0306,0.0246,0.0257,0.0249


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
        

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9584,0.8822,0.5268,0.959,0.68,0.6597,0.6938


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                

In [ ]:
상위 5개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

# prections = prep_pipe.predict_proba(AUTO_ML_test3)
# prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V

In [ ]:
# 이건 그냥 본거

scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157306
5,0.10,0.158258
6,0.12,0.291047
7,0.14,0.476756
8,0.16,0.641982
9,0.18,0.755834


In [ ]:
AUTO_ML_train3['model1_prob'] = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]

## (2)  model2

In [ ]:
AUTO_ML_train4 = AUTO_ML_train3.drop(['Y_LABEL', 'AL', 'BA'], axis = 1)

print(AUTO_ML_train4.shape)
print(AUTO_ML_test3.shape)

(14095, 19)
(6041, 18)


In [ ]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
print(len(categorical_features))
print(len(numeric_features))

2
16


In [ ]:
from pycaret.regression import *

clf = setup(AUTO_ML_train4, 
            target = 'model1_prob', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,3336
1,Target,model1_prob
2,Original Data,"(14095, 19)"
3,Missing Values,False
4,Numeric Features,16
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(9866, 36)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                                  

In [ ]:
best5 = compare_models(fold = 5, n_select = 5) 
# MAE -> 과대적합을 줄이기 위해
# CATBOOST를 선택한 이유
# 1. 표본의 잔차를 모두 다른 모형으로부터 구함으로써 과대적합을 피한다.
# 2. 특성변수가 범주형일 때 일반적으로 사용하는 one-hot encoding 대신 
#    실수인 순서목표통계량(ordered target statistic)으로 전환하여 사용한다. -> YEAR 변수에 잘 먹힐 것이라 생각함
# YEAR가 큰 값일수록 불량률이 낮았기 때문에, 실수형, 범주형 애매하다. 이건 순서형이다. 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0720,0.0245,0.1561,0.0716,0.1076,0.3547,6.588
gbr,Gradient Boosting Regressor,0.0716,0.0247,0.1570,0.0611,0.1076,0.3494,1.314
lightgbm,Light Gradient Boosting Machine,0.0726,0.0248,0.1574,0.0564,0.1086,0.3567,0.226
br,Bayesian Ridge,0.0746,0.0257,0.1602,0.0223,0.1097,0.3705,0.056
ridge,Ridge Regression,0.0744,0.0258,0.1602,0.0215,0.1098,0.3675,0.042
lr,Linear Regression,0.0744,0.0258,0.1602,0.0214,0.1098,0.3675,0.046
lar,Least Angle Regression,0.0744,0.0258,0.1603,0.0214,0.1098,0.3677,0.050
rf,Random Forest Regressor,0.0766,0.0257,0.1602,0.0214,0.1116,0.3936,7.942
omp,Orthogonal Matching Pursuit,0.0753,0.0262,0.1615,0.0058,0.1109,0.3765,0.044
llar,Lasso Least Angle Regression,0.0773,0.0263,0.1620,-0.0007,0.1116,0.3964,0.042


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 2
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7f8540525850>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=3336, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
        

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0740,0.0264,0.1625,0.0545,0.1111,0.3502
1,0.0743,0.0279,0.1672,0.0724,0.1134,0.3408
2,0.0736,0.0252,0.1587,0.0572,0.1087,0.3579
3,0.0679,0.0211,0.1453,0.1039,0.0996,0.3448
4,0.0680,0.0212,0.1457,0.0834,0.1003,0.3507
Mean,0.0716,0.0244,0.1559,0.0743,0.1066,0.3489
Std,0.0030,0.0028,0.0089,0.0182,0.0056,0.0058


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7f85405a3090>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0717,0.025,0.1582,0.0772,0.1081,0.343


In [ ]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7f85405a3090>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_sample

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

prections = prep_pipe.predict(AUTO_ML_test3)
prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                     

array([0.14124112, 0.17762949, 0.20040046, ..., 0.18727131, 0.15315545,
       0.12057331])

In [ ]:
scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict(AUTO_ML_train4)
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

# 0.22

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157296
5,0.10,0.157543
6,0.12,0.161319
7,0.14,0.193473
8,0.16,0.263111
9,0.18,0.348857


In [ ]:
answer = np.zeros(prections.shape[0])

for i in range(prections.shape[0]) :
  if prections[i] >= 0.22 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5609, 1.0: 432})

In [ ]:
# 제출

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_pycaret4.csv', index = False)

# 6. train 데이터에서 AL, BA, S 사용하여 AutoML(변수 21개) 그리고
#    knowledge distillation(변수 18개)

## (1)  model1

In [ ]:
new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']
# 주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개
# 부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개
# 부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개

In [11]:
AUTO_ML_train1 = Rdata_train.copy()
AUTO_ML_test1 = Rdata_test.copy()

In [13]:
AUTO_ML_train2 = AUTO_ML_train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA', 'S']]
AUTO_ML_test2 = AUTO_ML_test1.drop(['ID'], axis = 1)

print(AUTO_ML_train2.shape) # 변수 21개
print(AUTO_ML_test2.shape) # 변수 18개

(14095, 22)
(6041, 18)


In [14]:
le1 = LabelEncoder()
le2 = LabelEncoder()

AUTO_ML_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(AUTO_ML_train2['COMPONENT_ARBITRARY'])
AUTO_ML_train2['YEAR_category'] = le2.fit_transform(AUTO_ML_train2['YEAR'])

AUTO_ML_test2['COMPONENT_ARBITRARY_category'] = le1.transform(AUTO_ML_test2['COMPONENT_ARBITRARY'])
AUTO_ML_test2['YEAR_category'] = le2.transform(AUTO_ML_test2['YEAR'])

AUTO_ML_train3 = AUTO_ML_train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
AUTO_ML_test3 = AUTO_ML_test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [15]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL', 'BA', 'S']

print(AUTO_ML_train3.shape)
print(AUTO_ML_test3.shape)

(14095, 22)
(6041, 18)


In [16]:
from pycaret.classification import *

clf = setup(AUTO_ML_train3, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,4530
1,Target,Y_LABEL
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14095, 22)"
5,Missing Values,False
6,Numeric Features,19
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
        

In [17]:
best5 = compare_models(fold = 5, sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9530,0.8727,0.5069,0.9276,0.6547,0.6319,0.6657,1.896
catboost,CatBoost Classifier,0.9539,0.8706,0.5126,0.9359,0.6611,0.6388,0.6729,8.984
lightgbm,Light Gradient Boosting Machine,0.9527,0.8630,0.5103,0.9165,0.6548,0.6317,0.6634,0.410
rf,Random Forest Classifier,0.9536,0.8551,0.4966,0.9569,0.6525,0.6305,0.6702,1.532
ada,Ada Boost Classifier,0.9518,0.8545,0.4886,0.9335,0.6404,0.6173,0.6552,0.556
lr,Logistic Regression,0.9510,0.8507,0.4702,0.9508,0.6281,0.6052,0.6490,1.178
et,Extra Trees Classifier,0.9476,0.8478,0.4232,0.9642,0.5867,0.5634,0.6190,1.162
lda,Linear Discriminant Analysis,0.9244,0.8268,0.1640,0.8976,0.2768,0.2565,0.3645,0.094
nb,Naive Bayes,0.8985,0.8201,0.5827,0.4549,0.5073,0.4525,0.4587,0.050
dt,Decision Tree Classifier,0.9157,0.7554,0.5608,0.5216,0.5397,0.4934,0.4942,0.138


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=4530, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), <catboost.core.CatBoostClassifier object at 0x7fe1a5991810>, LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learnin

In [18]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'AUC')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9544,0.8520,0.5143,0.9474,0.6667,0.6445,0.6793
1,0.9493,0.8633,0.4540,0.9405,0.6124,0.5888,0.6338
2,0.9503,0.8859,0.4770,0.9222,0.6288,0.6050,0.6430
3,0.9645,0.8947,0.6092,0.9815,0.7518,0.7338,0.7581
4,0.9534,0.8812,0.5029,0.9462,0.6567,0.6342,0.6709
Mean,0.9544,0.8754,0.5115,0.9476,0.6633,0.6413,0.6770
Std,0.0054,0.0156,0.0531,0.0192,0.0483,0.0504,0.0439


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
        

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9577,0.885,0.4924,0.9368,0.6455,0.6253,0.6621


In [19]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                

In [21]:
# 상위 5개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

# prections = prep_pipe.predict_proba(AUTO_ML_test3)
# prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V

In [ ]:
# 이건 그냥 본거

scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157306
5,0.10,0.158258
6,0.12,0.291047
7,0.14,0.476756
8,0.16,0.641982
9,0.18,0.755834


In [22]:
AUTO_ML_train3['model1_prob'] = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]

## (2)  model2

In [23]:
AUTO_ML_train4 = AUTO_ML_train3.drop(['Y_LABEL', 'AL', 'BA', 'S'], axis = 1)

print(AUTO_ML_train4.shape)
print(AUTO_ML_test3.shape)

(14095, 19)
(6041, 18)


In [24]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
print(len(categorical_features))
print(len(numeric_features))

2
16


In [25]:
from pycaret.regression import *

clf = setup(AUTO_ML_train4, 
            target = 'model1_prob', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,7550
1,Target,model1_prob
2,Original Data,"(14095, 19)"
3,Missing Values,False
4,Numeric Features,16
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(9866, 36)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                                  

In [26]:
best5 = compare_models(fold = 5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.0802,0.0265,0.1626,0.0728,0.1126,0.4029,1.610
catboost,CatBoost Regressor,0.0807,0.0265,0.1627,0.0714,0.1133,0.4096,5.484
lightgbm,Light Gradient Boosting Machine,0.0806,0.0269,0.1638,0.0579,0.1142,0.4059,0.226
br,Bayesian Ridge,0.0835,0.0276,0.1662,0.0313,0.1151,0.4274,0.054
lr,Linear Regression,0.0832,0.0277,0.1662,0.0306,0.1152,0.4244,0.066
ridge,Ridge Regression,0.0832,0.0277,0.1662,0.0306,0.1152,0.4244,0.040
lar,Least Angle Regression,0.0832,0.0277,0.1662,0.0306,0.1152,0.4244,0.046
rf,Random Forest Regressor,0.0859,0.0279,0.1669,0.0231,0.1175,0.4557,7.574
omp,Orthogonal Matching Pursuit,0.0847,0.0282,0.1678,0.0115,0.1165,0.4379,0.040
llar,Lasso Least Angle Regression,0.0867,0.0285,0.1688,-0.0004,0.1174,0.4570,0.040


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 2
INFO:logs:[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7550, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), <catboost.core.CatBoostRegressor object at 0x7fe1a5861990>, LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
        

In [27]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0823,0.0292,0.1708,0.0624,0.1175,0.3943
1,0.0784,0.0239,0.1546,0.1145,0.1069,0.4106
2,0.0784,0.0239,0.1545,0.0912,0.1072,0.4109
3,0.0815,0.0269,0.1640,0.0801,0.1137,0.3990
4,0.0821,0.0277,0.1665,0.0431,0.1152,0.4126
Mean,0.0806,0.0263,0.1621,0.0782,0.1121,0.4055
Std,0.0018,0.0021,0.0065,0.0244,0.0043,0.0074


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
     

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0823,0.028,0.1674,0.0712,0.1153,0.4071


In [28]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                             

In [29]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

prections = prep_pipe.predict(AUTO_ML_test3)
prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                     

array([0.1410141 , 0.18464798, 0.19426746, ..., 0.18426805, 0.15239424,
       0.12762391])

In [30]:
scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict(AUTO_ML_train4)
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

# 0.22

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157286
5,0.10,0.157543
6,0.12,0.162392
7,0.14,0.198365
8,0.16,0.268496
9,0.18,0.358313


In [31]:
answer = np.zeros(prections.shape[0])

for i in range(prections.shape[0]) :
  if prections[i] >= 0.22 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5576, 1.0: 465})

In [32]:
# 제출

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_pycaret5.csv', index = False)

# 7. train 데이터에서 AL, BA, S, SI 사용하여 AutoML(변수 22개) 그리고
#    knowledge distillation(변수 18개)

## (1)  model1

In [ ]:
new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']
# 주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개
# 부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개
# 부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개

In [33]:
AUTO_ML_train1 = Rdata_train.copy()
AUTO_ML_test1 = Rdata_test.copy()

In [35]:
AUTO_ML_train2 = AUTO_ML_train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA', 'S', 'SI']]
AUTO_ML_test2 = AUTO_ML_test1.drop(['ID'], axis = 1)

print(AUTO_ML_train2.shape) # 변수 22개
print(AUTO_ML_test2.shape) # 변수 18개

(14095, 23)
(6041, 18)


In [36]:
le1 = LabelEncoder()
le2 = LabelEncoder()

AUTO_ML_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(AUTO_ML_train2['COMPONENT_ARBITRARY'])
AUTO_ML_train2['YEAR_category'] = le2.fit_transform(AUTO_ML_train2['YEAR'])

AUTO_ML_test2['COMPONENT_ARBITRARY_category'] = le1.transform(AUTO_ML_test2['COMPONENT_ARBITRARY'])
AUTO_ML_test2['YEAR_category'] = le2.transform(AUTO_ML_test2['YEAR'])

AUTO_ML_train3 = AUTO_ML_train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
AUTO_ML_test3 = AUTO_ML_test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [37]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL', 'BA', 'S', 'SI']

print(AUTO_ML_train3.shape)
print(AUTO_ML_test3.shape)

(14095, 23)
(6041, 18)


In [38]:
from pycaret.classification import *

clf = setup(AUTO_ML_train3, 
            target = 'Y_LABEL', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,886
1,Target,Y_LABEL
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(14095, 23)"
5,Missing Values,False
6,Numeric Features,20
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
        

In [39]:
best5 = compare_models(fold = 5, sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9535,0.8711,0.4953,0.9275,0.6455,0.6230,0.6584,9.128
gbc,Gradient Boosting Classifier,0.9523,0.8710,0.4917,0.9088,0.6380,0.6148,0.6483,1.988
lightgbm,Light Gradient Boosting Machine,0.9532,0.8671,0.4988,0.9159,0.6456,0.6229,0.6561,0.454
ada,Ada Boost Classifier,0.9519,0.8643,0.4810,0.9172,0.6306,0.6076,0.6442,0.578
rf,Random Forest Classifier,0.9540,0.8564,0.4846,0.9560,0.6428,0.6210,0.6624,1.580
lr,Logistic Regression,0.9510,0.8478,0.4514,0.9503,0.6119,0.5893,0.6362,0.824
et,Extra Trees Classifier,0.9475,0.8432,0.3993,0.9687,0.5652,0.5423,0.6035,1.154
nb,Naive Bayes,0.8738,0.8154,0.5972,0.3620,0.4487,0.3829,0.3993,0.050
lda,Linear Discriminant Analysis,0.9254,0.8113,0.1410,0.9249,0.2444,0.2267,0.3444,0.094
dt,Decision Tree Classifier,0.9159,0.7473,0.5439,0.5087,0.5254,0.4793,0.4798,0.152


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 2
INFO:logs:[<catboost.core.CatBoostClassifier object at 0x7fe19bad1f90>, GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=886, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning

In [40]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'AUC')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9519,0.8749,0.4793,0.9205,0.6304,0.6073,0.6445
1,0.9513,0.8861,0.4821,0.9000,0.6279,0.6044,0.6383
2,0.9534,0.8784,0.4852,0.9425,0.6406,0.6184,0.6576
3,0.9554,0.8741,0.4911,0.9765,0.6535,0.6325,0.6754
4,0.9569,0.8759,0.5207,0.9565,0.6743,0.6534,0.6882
Mean,0.9538,0.8779,0.4917,0.9392,0.6454,0.6232,0.6608
Std,0.0021,0.0044,0.0150,0.0268,0.0171,0.0180,0.0187


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7fe1a5a52e50>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9563,0.8797,0.5292,0.9223,0.6726,0.651,0.6799


In [41]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7fe1a5a52e50>),
                             ('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_spl

In [42]:
# 상위 5개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

# prections = prep_pipe.predict_proba(AUTO_ML_test3)
# prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V

In [ ]:
# 이건 그냥 본거

scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157306
5,0.10,0.158258
6,0.12,0.291047
7,0.14,0.476756
8,0.16,0.641982
9,0.18,0.755834


In [43]:
AUTO_ML_train3['model1_prob'] = prep_pipe.predict_proba(AUTO_ML_train3)[:, 1]

## (2)  model2

In [44]:
AUTO_ML_train4 = AUTO_ML_train3.drop(['Y_LABEL', 'AL', 'BA', 'S', 'SI'], axis = 1)

print(AUTO_ML_train4.shape)
print(AUTO_ML_test3.shape)

(14095, 19)
(6041, 18)


In [45]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
numeric_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
                    'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
print(len(categorical_features))
print(len(numeric_features))

2
16


In [46]:
from pycaret.regression import *

clf = setup(AUTO_ML_train4, 
            target = 'model1_prob', 
            categorical_features = categorical_features,
            numeric_features = numeric_features,
            normalize = True,
            iterative_imputation_iters = 10,
            train_size = 0.7)

,Description,Value
0,session_id,6711
1,Target,model1_prob
2,Original Data,"(14095, 19)"
3,Missing Values,False
4,Numeric Features,16
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(9866, 36)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                                  

In [47]:
best5 = compare_models(fold = 5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0845,0.0282,0.1678,0.0577,0.1168,0.4270,5.344
gbr,Gradient Boosting Regressor,0.0844,0.0282,0.1680,0.0556,0.1163,0.4234,1.488
lightgbm,Light Gradient Boosting Machine,0.0841,0.0284,0.1685,0.0495,0.1174,0.4219,0.232
rf,Random Forest Regressor,0.0899,0.0295,0.1718,0.0125,0.1207,0.4752,7.460
br,Bayesian Ridge,0.0875,0.0295,0.1718,0.0114,0.1187,0.4491,0.072
ridge,Ridge Regression,0.0873,0.0296,0.1720,0.0096,0.1188,0.4464,0.064
lr,Linear Regression,0.0873,0.0296,0.1720,0.0095,0.1188,0.4464,0.048
lar,Least Angle Regression,0.0873,0.0296,0.1720,0.0095,0.1188,0.4464,0.068
omp,Orthogonal Matching Pursuit,0.0883,0.0298,0.1726,0.0030,0.1195,0.4559,0.060
llar,Lasso Least Angle Regression,0.0897,0.0299,0.1729,-0.0002,0.1201,0.4714,0.064


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 2
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7fe19b992190>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=6711, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
        

In [48]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0848,0.0284,0.1686,0.0733,0.1162,0.4206
1,0.0849,0.0284,0.1686,0.0994,0.1164,0.4172
2,0.0840,0.0256,0.1600,0.0680,0.1120,0.4348
3,0.0856,0.0288,0.1696,0.0526,0.1171,0.4392
4,0.0847,0.0278,0.1666,0.0593,0.1155,0.4353
Mean,0.0848,0.0278,0.1667,0.0705,0.1154,0.4294
Std,0.0005,0.0011,0.0035,0.0161,0.0018,0.0088


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 3
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe19b9dc4d0>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0799,0.0244,0.1562,0.0967,0.1084,0.4206


In [49]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7fe19b9dc4d0>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_sample

In [50]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])

prections = prep_pipe.predict(AUTO_ML_test3)
prections

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['COMPONENT_ARBITRARY_category',
                                                            'YEAR_category'],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['ANONYMOUS_1',
                                                          'ANONYMOUS_2', 'AG',
                                                          'CO', 'CR', 'CU',
                                                          'FE', 'H2O', 'MN',
                                                          'MO', 'NI', 'PQINDEX',
                                                          'TI', 'V', 'V40',
                                     

array([0.13352878, 0.17329997, 0.18980368, ..., 0.21955356, 0.14926843,
       0.12971068])

In [51]:
scores = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = prep_pipe.predict(AUTO_ML_train4)
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(AUTO_ML_train3['Y_LABEL'], pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores

# 0.22

,threshold,score
0,0.00,0.157275
1,0.02,0.157275
2,0.04,0.157275
3,0.06,0.157275
4,0.08,0.157275
5,0.10,0.157481
6,0.12,0.168487
7,0.14,0.225268
8,0.16,0.333715
9,0.18,0.475985


In [55]:
answer = np.zeros(prections.shape[0])

for i in range(prections.shape[0]) :
  if prections[i] >= 0.24 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5667, 1.0: 374})

In [56]:
# 제출

submission_preds = answer
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/건설기계 오일 상태 분류 AI 경진대회/결과/submission_pycaret6.csv', index = False)